In [1]:
from google.colab import files
import json

# Load the data
with open('train.json', 'r') as f:
    data = json.load(f)

print("Original data:")
print(json.dumps(data, indent=2, ensure_ascii=False)[:500])  # Show preview

# Function to clean the data
def drop_fields(entries, fields):
    """Remove specified fields from each entry."""
    field_set = set(fields)

    for entry in entries.values():
        if not isinstance(entry, dict):
            continue
        for field in field_set:
            entry.pop(field, None)

# List of fields to remove
fields_to_remove = ["choices", "average", "stdev", "nonsensical", "sample_id"]

# Create cleaned data (make a copy to preserve original if needed)
import copy
data_clean = copy.deepcopy(data)

# Clean the data
drop_fields(data_clean, fields_to_remove)

print("\n" + "="*50)
print("Cleaned data:")
print(json.dumps(data_clean, indent=4, ensure_ascii=False))

print(f"\n✓ data_clean is ready to use!")

Original data:
{
  "0": {
    "homonym": "potential",
    "judged_meaning": "the difference in electrical charge between two points in a circuit expressed in volts",
    "precontext": "The old machine hummed in the corner of the workshop. Clara examined its dusty dials with a furrowed brow. She wondered if it could be brought back to life.",
    "sentence": "The potential couldn't be measured.",
    "ending": "She collected a battery reader and looked on earnestly, willing some life back into the old machine."

Cleaned data:
{
    "0": {
        "homonym": "potential",
        "judged_meaning": "the difference in electrical charge between two points in a circuit expressed in volts",
        "precontext": "The old machine hummed in the corner of the workshop. Clara examined its dusty dials with a furrowed brow. She wondered if it could be brought back to life.",
        "sentence": "The potential couldn't be measured.",
        "ending": "She collected a battery reader and looked on ear

In [2]:
# Minify the cleaned data
def minify_json_data(data):
    """Convert data to minified JSON string (no whitespace)."""
    return json.dumps(data, separators=(",", ":"), ensure_ascii=False)

# Create minified version
data_cleaned_minified = minify_json_data(data_clean)

# Save to file
with open('data_cleaned_minified.json', 'w', encoding='utf-8') as f:
    f.write(data_cleaned_minified)

print("✓ data_cleaned_minified.json created!")
print(f"\nFile size: {len(data_cleaned_minified)} characters")
print(f"\nPreview (first 200 chars):\n{data_cleaned_minified[:200]}...")

✓ data_cleaned_minified.json created!

File size: 1105313 characters

Preview (first 200 chars):
{"0":{"homonym":"potential","judged_meaning":"the difference in electrical charge between two points in a circuit expressed in volts","precontext":"The old machine hummed in the corner of the workshop...


In [3]:
# Create solution.jsonl from original data
def create_solution_file(data, output_path='solution.jsonl'):
    """
    Extract id and choices (labels) from data and create a JSONL file.
    Each line contains one JSON object with 'id' and 'label' fields.
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        for sample_id, entry in data.items():
            # Extract choices as label
            if isinstance(entry, dict) and 'choices' in entry:
                solution_entry = {
                    "id": sample_id,
                    "label": entry['choices']
                }
                # Write one JSON object per line
                f.write(json.dumps(solution_entry, ensure_ascii=False) + '\n')

    print(f"✓ {output_path} created!")

# Generate the solution file
create_solution_file(data)

# Display preview
print("\nPreview of solution.jsonl:")
with open('solution.jsonl', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i < 5:  # Show first 5 lines
            print(line.strip())
        else:
            break
print("...")

✓ solution.jsonl created!

Preview of solution.jsonl:
{"id": "0", "label": [4, 5, 2, 3, 1]}
{"id": "1", "label": [5, 3, 4, 4, 3]}
{"id": "2", "label": [2, 1, 4, 3, 1]}
{"id": "3", "label": [4, 5, 5, 3, 5]}
{"id": "4", "label": [1, 1, 4, 4, 3]}
...


In [4]:
import json
import pandas as pd
import random

# ---- CONFIG ----
SHUFFLE_LABELS = True  # Shuffle annotation order (order not important)
RANDOM_SEED = 42       # For reproducibility

def build_input(row):
    """Format row data into LLM input text."""
    return (
        f"Homonym: {row.get('homonym', '')}\n"
        f"Judged meaning: {row.get('judged_meaning', '')}\n"
        f"Context: {row.get('precontext', '')}\n"
        f"Sentence: {row.get('sentence', '')}\n"
        f"Ending: {row.get('ending', '')}\n"
        f"Example meaning sentence: {row.get('example_sentence', '')}"
    ).strip()

def expand_annotations(merged_df, shuffle=True, seed=42):
    """
    Create a training sample for EACH individual annotation.

    Args:
        merged_df: DataFrame with columns input_text and label (list)
        shuffle: If True, shuffle annotation order in each list
        seed: Seed for reproducibility

    Returns:
        List of dictionaries with text and individual label
    """
    if seed:
        random.seed(seed)

    output_records = []

    for _, row in merged_df.iterrows():
        text = row['input_text']
        labels = row['label']

        # Check that label is a list
        if not isinstance(labels, list):
            labels = [labels]

        # Filter valid values (1-5)
        valid_labels = [l for l in labels if isinstance(l, (int, float)) and 1 <= l <= 5]

        if not valid_labels:
            continue

        # Optional: shuffle annotation order
        if shuffle:
            valid_labels = valid_labels.copy()
            random.shuffle(valid_labels)

        # Create a sample for each annotation
        for label in valid_labels:
            output_records.append({
                "text": text,
                "label": float(label)
            })

    return output_records

# Load minified cleaned data
with open('data_cleaned_minified.json', 'r', encoding='utf-8') as f:
    data_cleaned_minified = json.load(f)

dev_df = pd.DataFrame.from_dict(data_cleaned_minified, orient="index")
# Reset index to make sample_id a column
dev_df = dev_df.reset_index().rename(columns={'index': 'sample_id'})
print(f"✓ Loaded DEV set: {len(dev_df)} rows")

# Load solution data
sol_df = pd.read_json('solution.jsonl', lines=True)
sol_df = sol_df.rename(columns={"id": "sample_id"})
print(f"✓ Loaded solution set: {len(sol_df)} rows")

# Convert join keys to string
dev_df["sample_id"] = dev_df["sample_id"].astype(str)
sol_df["sample_id"] = sol_df["sample_id"].astype(str)

# Merge datasets
merged = dev_df.merge(sol_df, on="sample_id", how="inner")
print(f"✓ Merged dataset: {len(merged)} samples")

# Warn about lost samples
lost = len(dev_df) - len(merged)
if lost > 0:
    print(f"⚠ Warning: lost {lost} samples during merge ({lost/len(dev_df)*100:.1f}%)")

# Build text fields
merged["input_text"] = merged.apply(build_input, axis=1)

# Expand: create one sample per annotation
output_records = expand_annotations(
    merged,
    shuffle=SHUFFLE_LABELS,
    seed=RANDOM_SEED
)

print(f"✓ Expanded to {len(output_records)} training samples from {len(merged)} unique texts")
print(f"  → Average {len(output_records)/len(merged):.1f} annotations per sample")

# Save output
with open('train_llm_regression.jsonl', "w", encoding="utf-8") as f:
    for record in output_records:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print(f"✓ Saved {len(output_records)} samples → train_llm_regression.jsonl")

# Display preview
print("\nPreview of train_llm_regression.jsonl:")
with open('train_llm_regression.jsonl', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i < 3:  # Show first 3 lines
            print(json.loads(line))
        else:
            break

✓ Loaded DEV set: 2280 rows
✓ Loaded solution set: 2280 rows
✓ Merged dataset: 2280 samples
✓ Expanded to 11434 training samples from 2280 unique texts
  → Average 5.0 annotations per sample
✓ Saved 11434 samples → train_llm_regression.jsonl

Preview of train_llm_regression.jsonl:
{'text': "Homonym: potential\nJudged meaning: the difference in electrical charge between two points in a circuit expressed in volts\nContext: The old machine hummed in the corner of the workshop. Clara examined its dusty dials with a furrowed brow. She wondered if it could be brought back to life.\nSentence: The potential couldn't be measured.\nEnding: She collected a battery reader and looked on earnestly, willing some life back into the old machine.\nExample meaning sentence: The circuit has a high potential difference.", 'label': 3.0}
{'text': "Homonym: potential\nJudged meaning: the difference in electrical charge between two points in a circuit expressed in volts\nContext: The old machine hummed in the 

In [5]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-hz3nel4i/unsloth_c94375d51d8f46d1b800c1f21ab4b1bd
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-hz3nel4i/unsloth_c94375d51d8f46d1b800c1f21ab4b1bd
  Resolved https://github.com/unslothai/unsloth.git to commit 521e20127dac99a845abc78cb95d13040ace6616
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached xformers-0.0.26.post1.tar.gz (4.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.8.6-py3-none-any.whl (245 kB)
ERROR: Operation cancelled by user


In [6]:
!pip install modelscope

In [5]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'


from unsloth import FastLanguageModel
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# -------------------------
# 1. Configuration
# -------------------------
max_seq_length = 512 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# -------------------------
# 2. Load Optimized Model
# -------------------------
model, tokenizer = FastLanguageModel.from_pretrained(

    model_name = "meta-llama/Llama-3.2-3B-Instruct", # Use the 3B model for speed on Colab
    # this model literally takes 2,5 hours
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# -------------------------
# 3. Add LoRA Adapters (The "Training" Part)
# -------------------------
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

# -------------------------
# 4. Format Data
# -------------------------
# We need to format the input to look like a standard chat or text completion
# Since we are doing regression via "float training", we format the target in the text.
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Evaluate the homonym in the sentence and assign a probability score from 1.0 to 5.0.

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    inputs       = examples["text"] # From your jsonl
    outputs      = examples["label"] # From your jsonl
    texts = []
    for input_text, output_label in zip(inputs, outputs):
        # We format the float label as a string for the model to learn to generate
        text = alpaca_prompt.format(input_text, str(output_label)) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

# Load your JSONL
dataset = load_dataset("json", data_files="train_llm_regression.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

# -------------------------
# 5. Train (The Fast Part)
# -------------------------
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 60,  <-- DELETE or COMMENT OUT THIS LINE
        num_train_epochs = 4, # <-- ADD THIS LINE (Full pass over data)
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10, # Log every 10 steps so output isn't spammy
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer.train()

# -------------------------
# 6. Save
# -------------------------
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving













# Install required package
"""
!pip install -q bitsandbytes accelerate

import json
import os
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

from transformers import AutoTokenizer, AutoModel
from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error


# -------------------------
# Dataset
# -------------------------
class JsonlRegressionDataset(Dataset):
    def __init__(self, jsonl_path):
        self.samples = []
        with open(jsonl_path, "r", encoding="utf-8") as f:
            for line in f:
                data = json.loads(line.strip())
                self.samples.append({"text": data["text"], "label": float(data["label"])})

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


# -------------------------
# Regressor head
# -------------------------
class Regressor(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, 512),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1)
        )
    def forward(self, x):
        return self.net(x).squeeze(-1)


# -------------------------
# Mean pooling
# -------------------------
def mean_pooling(last_hidden_state, attention_mask):
    mask = attention_mask.unsqueeze(-1).float()
    return torch.sum(last_hidden_state * mask, dim=1) / torch.clamp(mask.sum(dim=1), min=1e-9)


# -------------------------
# Evaluation
# -------------------------
@torch.no_grad()
def evaluate(embedder, regressor, loader, device):
    embedder.eval(); regressor.eval()
    preds, trues = [], []
    for batch in loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = embedder(input_ids=input_ids, attention_mask=attention_mask)
        embeds = mean_pooling(outputs.last_hidden_state, attention_mask)
        pred = regressor(embeds)

        preds.extend(pred.cpu().numpy().tolist())
        trues.extend(labels.cpu().numpy().tolist())

    mse = mean_squared_error(trues, preds)
    spearman = spearmanr(trues, preds).correlation
    return mse, spearman


# -------------------------
# Configuration
# -------------------------
TRAIN_JSONL = "train_llm_regression.jsonl"
MODEL_NAME = "unsloth/llama-3.1-8b-instruct-bnb-4bit"
OUTPUT_DIR = "models/llama_regressor_unsloth"
BATCH_SIZE = 28
EPOCHS = 4
LR = 3e-4
MAX_LENGTH = 512

os.makedirs(OUTPUT_DIR, exist_ok=True)

# -------------------------
# Load tokenizer and model
# -------------------------
print("Loading tokenizer and model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Model (already 4-bit quantized)
embedder = AutoModel.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True
)
embedder.eval()
for p in embedder.parameters():
    p.requires_grad = False

# -------------------------
# Dataset preparation
# -------------------------
print("Loading dataset...")
dataset = JsonlRegressionDataset(TRAIN_JSONL)
val_size = int(0.1 * len(dataset))
train_ds, val_ds = random_split(dataset, [len(dataset) - val_size, val_size])

def collate_fn(batch):
    texts = [x["text"] for x in batch]
    labels = [x["label"] for x in batch]
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=MAX_LENGTH,
        padding=True,
        return_tensors="pt"
    )
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "labels": torch.tensor(labels, dtype=torch.float32)
    }

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# -------------------------
# Get embedding dimension
# -------------------------
sample = next(iter(train_loader))
with torch.no_grad():
    dim = embedder(**{k: v.to(embedder.device) for k, v in sample.items() if k != "labels"}).last_hidden_state.shape[-1]

# -------------------------
# Initialize regressor
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
regressor = Regressor(dim).to(device)
optimizer = torch.optim.AdamW(regressor.parameters(), lr=LR)
loss_fn = nn.MSELoss()

best_spearman = -1.0
print(f"Training on {len(train_ds)} samples | Val: {len(val_ds)} | Embedding dim: {dim}")

# -------------------------
# Training loop
# -------------------------
for epoch in range(1, EPOCHS + 1):
    regressor.train()
    losses = []
    for batch in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch.pop("labels")

        with torch.no_grad():
            out = embedder(**batch)
            embeds = mean_pooling(out.last_hidden_state, batch["attention_mask"])

        preds = regressor(embeds)
        loss = loss_fn(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

    val_mse, val_spearman = evaluate(embedder, regressor, val_loader, device)
    print(f"Epoch {epoch} | Loss: {np.mean(losses):.4f} | Val MSE: {val_mse:.4f} | Spearman: {val_spearman:.4f}")

    if val_spearman > best_spearman:
        best_spearman = val_spearman
        torch.save(regressor.state_dict(), os.path.join(OUTPUT_DIR, "regressor_best.pt"))
        with open(os.path.join(OUTPUT_DIR, "meta.json"), "w") as f:
            json.dump({"model_name": MODEL_NAME, "spearman": best_spearman}, f)
        print(f"✓ NEW BEST → Spearman = {best_spearman:.4f}")

print(f"\n✓ Training complete! Best Spearman: {best_spearman:.4f}")
"""

==((====))==  Unsloth 2025.12.1: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth


Unsloth 2025.12.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/11434 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/11434 [00:00<?, ? examples/s]

...

Unsloth: Will smartly offload gradients to save VRAM!


wandb: WARNING URL not available in offline run
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:295: UserWarning: Could not find a config file in unsloth/llama-3.2-3b-instruct-unsloth-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(
wandb: WARNING URL not available in offline run
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:295: UserWarning: Could not find a config file in unsloth/llama-3.2-3b-instruct-unsloth-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(
wandb: WARNING URL not available in offline run
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:295: UserWarning: Could not find a config file in unsloth/llama-3.2-3b-instruct-unsloth-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(
wandb: WARNING URL not available in offline run
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:295: UserWarning: Could not find a con

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇█
train/global_step,▁▁▁▁▁▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇█
train/grad_norm,▆▆▆█▅▅▃▄▁▂▂▂▁▃▂▁▁▁▁▅▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,█▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁
train/loss,█▄▄▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,1.314516433955881e+17
train/epoch,4
train/global_step,5720
train/grad_norm,0.21245
train/learning_rate,0.0
train/loss,0.0593


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:295: UserWarning: Could not find a config file in unsloth/llama-3.2-3b-instruct-unsloth-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(


'\n!pip install -q bitsandbytes accelerate\n\nimport json\nimport os\nimport numpy as np\nfrom tqdm import tqdm\n\nimport torch\nimport torch.nn as nn\nfrom torch.utils.data import Dataset, DataLoader, random_split\n\nfrom transformers import AutoTokenizer, AutoModel\nfrom scipy.stats import spearmanr\nfrom sklearn.metrics import mean_squared_error\n\n\n# -------------------------\n# Dataset\n# -------------------------\nclass JsonlRegressionDataset(Dataset):\n    def __init__(self, jsonl_path):\n        self.samples = []\n        with open(jsonl_path, "r", encoding="utf-8") as f:\n            for line in f:\n                data = json.loads(line.strip())\n                self.samples.append({"text": data["text"], "label": float(data["label"])})\n\n    def __len__(self):\n        return len(self.samples)\n\n    def __getitem__(self, idx):\n        return self.samples[idx]\n\n\n# -------------------------\n# Regressor head\n# -------------------------\nclass Regressor(nn.Module):\n    

In [ ]:
# Get your HuggingFace token from https://huggingface.co/settings/tokens
# You need a WRITE token

model.push_to_hub(
    "afafasdfasfa/wsd-llama3.2-lora",  # Replace with your HF username
    token = "...",  # Your HuggingFace token
)

tokenizer.push_to_hub(
    "afafasdfasfa/wsd-llama3.2-lora",
    token = "...",
)

print("✅ Model uploaded to HuggingFace!")

Saved model to https://huggingface.co/afafasdfasfa/wsd-llama3.2-lora
✅ Model uploaded to HuggingFace!
